In [1]:
from MetaWorld.utilsMW.trainer import ActiveCritic, ActiveCriticArgs
from stable_baselines3.common.torch_layers import CombinedExtractor
from MetaWorld.utilsMW.model_setup_obj import NetworkSetup
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv
import gym
from MetaWorld.searchTest.utils import MyEnv, ToyExpertModel, sample_expert_transitions, benchmark_policy, LearnWrapper, train_policy, LearnWrapper, parse_sampled_transitions, VecExtractor
import torch
import zipfile
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
import importlib
import numpy as np
from RlBaselines3Zoo import enjoy
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.ppo import MlpPolicy

from imitation.algorithms import bc
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper

from stable_baselines3.common.policies import MultiInputActorCriticPolicy, ActorCriticPolicy, BaseModel
from stable_baselines3.common.torch_layers import (
    BaseFeaturesExtractor,
    CombinedExtractor,
    FlattenExtractor,
    NatureCNN,
    create_mlp,
    get_actor_critic_arch,
)
from sb3_contrib.tqc.tqc import TQC
from sb3_contrib.tqc.policies import MultiInputPolicy
import numpy as np
import torch as th
import os

from MetaWorld.searchTest.utils import train_policy

import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

model = enjoy.main(inpt_args="--algo tqc --env FetchPickAndPlace-v1 --folder /home/hendrik/Documents/master_project/Code/RlBaselines3Zoo/rl-trained-agents -n 300 --ret_model True")
def sample_expert_transitions():
    expert = model

    print("Sampling expert transitions.")
    rollouts = rollout.rollout(
        expert,
        DummyVecEnv([lambda: RolloutInfoWrapper(model.env.envs[0])]),
        rollout.make_sample_until(min_timesteps=None, min_episodes=100),
        unwrap=True,
        exclude_infos=False,
    )
    return rollout.flatten_trajectories(rollouts)

tp = '/home/hendrik/Documents/master_project/LokalData/ImitationLearning/transitions_rew_100'

#transitions = sample_expert_transitions()
#torch.save(transitions, tp)
transitions = torch.load('/home/hendrik/Documents/master_project/LokalData/ImitationLearning/transitions_rew_100')



/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-08-30 18:00:47.719664: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-08-30 18:00:49.558166: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-08-30 18:00:49.558240: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-08-30 18:00:49.558324: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-30 18:00:49.558451: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Fou

1 Physical GPUs, 1 Logical GPUs
Loading latest experiment, id=1
Loading /home/hendrik/Documents/master_project/Code/RlBaselines3Zoo/rl-trained-agents/tqc/FetchPickAndPlace-v1_1/FetchPickAndPlace-v1.zip


In [2]:
from MetaWorld.metaworld.policies.sawyer_pick_place_v2_policy import SawyerPickPlaceV2Policy
from metaworld.envs import ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE
from MetaWorld.utilsMW.makeTrainingData import make_policy_dict
from stable_baselines3.common.policies import ActorCriticPolicy
from stable_baselines3.common.on_policy_algorithm import OnPolicyAlgorithm
from typing import Union, Type, Optional
from gym.wrappers import TimeLimit
from typing import Any

from stable_baselines3.common.type_aliases import GymEnv
policy_dict = make_policy_dict()
env_tag = 'reach'
gt_policy = policy_dict[env_tag]
pape = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict[env_tag][1]]()
pape._freeze_rand_vec = False
timelimit = TimeLimit(env=pape, max_episode_steps=2)
dv1 = DummyVecEnv([lambda: RolloutInfoWrapper(timelimit)])
class ImitationLearningWrapper:
    def __init__(self, policy, env:GymEnv):
        self.observation_space = env.observation_space
        self.action_space = env.action_space
        self.policy = policy

    def predict(self, obsv, deterministic=None):
        actions = []
        for obs in obsv:
            actions.append(self.policy.get_action(obs))
        return actions
IGTP = ImitationLearningWrapper(policy=gt_policy[0], env=dv1).predict

def new_epoch(current_obs, check_obsvs):
    result =  not th.equal(current_obs.reshape(-1)[-3:], check_obsvs.reshape(-1)[-3:])
    return result
    
class DummyExtractor:
    def __init__(self):
        pass
    def forward(self, features):
        if type(features) is np.ndarray:
            features = th.tensor(features)
        return features

def sample_expert_transitions(expert):
    print("Sampling expert transitions.")
    rollouts = rollout.rollout(
        expert,
        dv1,
        rollout.make_sample_until(min_timesteps=None, min_episodes=35),
        unwrap=True,
        exclude_infos=False,
    )
    return rollout.flatten_trajectories(rollouts)
#transitions = sample_expert_transitions(IGTP)

In [3]:
pape = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict[env_tag][1]]()
pape._freeze_rand_vec = False
timelimit = TimeLimit(env=pape, max_episode_steps=2)

In [6]:
from random import seed


class WholeSequenceWrapper(gym.Env):
    def __init__(self, env:gym.Env) -> None:
        super().__init__()
        asd = env()
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=[400], dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=0, high=2**32-1, shape=[1], dtype=np.int64)
        self.env = env

    def reset(self):
        self.seeds = np.random.randint(0,2**32-1, 1)
        return self.seeds

    def render(self, mode) -> Any:
        self.env.render(mode=mode)

    def step(self, action):# -> Tuple[_OperationType, float, bool, Dict[str, Any]]:
        action_dim = self.env().action_space.shape[0]
        actions = action.reshape([-1, action_dim])
        seeded_env = self.env(seed=self.seeds)
        for num_action in range(actions.shape[0]):
            obsv, reward, dones, info = seeded_env.step(actions[num_action])
        rew = info['success']
        return self.seeds, rew, dones, info

class WholeSequenceExpert(ImitationLearningWrapper):
    def __init__(self, policy, env: GymEnv, seq_len = 100):
        super().__init__(policy, env())
        self.env = env
        self.seq_len = seq_len

    def predict(self, obsv, deterministic=None):
        seeded_env = self.env(seed=obsv)
        timelimit = TimeLimit(env=seeded_env, max_episode_steps=self.seq_len)
        act_obsv = timelimit.reset()
        dones = False
        actions = []
        while not dones:
            next_action = self.policy.get_action(act_obsv)
            actions.append(next_action)
            act_obsv, reward, dones, info = timelimit.step(next_action)
        actions = np.array(actions).reshape([1,-1])
        return actions

class WholeSequenceExpertVec(WholeSequenceExpert):
    def __init__(self, policy, env: GymEnv, seq_len=100):
        super().__init__(policy, env, seq_len)

    def predict(self, obsv, state=None, episode_start=None, deterministic=None):
        obsv = int(obsv[0,0])
        result = super().predict(obsv, deterministic)
        return result

In [7]:
gt_policy = policy_dict[env_tag][0]
pape = ALL_V2_ENVIRONMENTS_GOAL_OBSERVABLE[policy_dict[env_tag][1]]
wse = WholeSequenceExpertVec(policy=gt_policy, env=pape, seq_len=100)
wsw = WholeSequenceWrapper(env=pape)
timelimit = TimeLimit(env=wsw, max_episode_steps=1)
wsdv = DummyVecEnv([lambda: RolloutInfoWrapper(timelimit)])
obsv = wsdv.reset()

In [9]:
def sample_expert_transitions(expert, env):
    print("Sampling expert transitions.")
    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=None, min_episodes=1000),
        unwrap=True,
        exclude_infos=False,
    )
    return rollout.flatten_trajectories(rollouts)
transitions = sample_expert_transitions(wse.predict, wsdv)
#transitions = torch.load('/tmp/transitions')

Sampling expert transitions.


In [10]:
torch.save(transitions, '/tmp/transitions')

In [11]:
transitions[0]['acts'].shape

(400,)

In [19]:
from stable_baselines3.common.policies import ActorCriticPolicy
net_arch = [dict(pi=[512, 512, 512], vf=[512, 512, 512])]
policy = ActorCriticPolicy(observation_space=wsdv.observation_space, action_space=wsdv.action_space, lr_schedule=lambda a:1, net_arch=net_arch)

In [20]:
pytorch_total_params = sum(p.numel() for p in policy.parameters())
pytorch_total_params

1258785

In [21]:
bc_trainer = bc.BC(
    policy=policy,
    observation_space=wsdv.observation_space,
    action_space=wsdv.action_space,
    demonstrations=transitions,
    device='cuda',
    optimizer_kwargs=dict({"lr":1e-4})
)


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [18]:
rewards = []
for i in range(50):
    obsv = wsdv.reset()
    action = bc_trainer.policy.predict(observation=obsv)
    a = wsdv.step(action)
    rewards.append(a[1])
mean_reward = np.array(rewards).mean()

IndentationError: unexpected indent (418903672.py, line 2)

In [15]:
ov_rew = []
for i in range(200):
    bc_trainer.train(n_epochs=500, progress_bar=False, log_interval=10000)
    rewards = []
    for i in range(50):
        obsv = wsdv.reset()
        action = bc_trainer.policy.predict(observation=obsv)
        a = wsdv.step(action)
        rewards.append(a[1])
    mean_reward = np.array(rewards).mean()
    print(mean_reward)
    ov_rew.append(mean_reward)
    

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.568    |
|    entropy        | 568       |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 2.05e+03  |
|    loss           | 381       |
|    neglogp        | 381       |
|    prob_true_act  | 1.42e-164 |
|    samples_so_far | 32        |
---------------------------------


/home/hendrik/anaconda3/envs/tfTest/lib/python3.9/site-packages/torch/utils/data/_utils/collate.py:149: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:178.)
  return default_collate([torch.as_tensor(b) for b in batch])


--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 10000    |
|    ent_loss       | -0.17    |
|    entropy        | 170      |
|    epoch          | 322      |
|    l2_loss        | 0        |
|    l2_norm        | 2.25e+03 |
|    loss           | -18.8    |
|    neglogp        | -18.7    |
|    prob_true_act  | 4.8e+11  |
|    samples_so_far | 320032   |
--------------------------------
0.1
---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | 0.0426    |
|    entropy        | -42.6     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 2.53e+03  |
|    loss           | -217      |
|    neglogp        | -217      |
|    prob_true_act  | 2.42e+102 |
|    samples_so_far | 32        |
---------------------------------
---------------------------------
| batch_size        | 32

KeyboardInterrupt: 

In [ ]:
action = bc_trainer.policy.predict(observation=obsv)

In [ ]:
action[0].shape

In [ ]:
a = wsdv.step(action)

In [ ]:
a[1]